In [1]:
from model import *

In [2]:
gin.parse_config_file("config/gpt2-small.gin")
config = GPTConfig()
print(config)

GPTConfig(block_size=1024, vocab_size=100288, n_layer=16, n_head=16, n_embd=1024, batch_size=4, epochs=2, device='cuda', clip_grad_norm_val=1.0, training_backend='nccl', learning_rate=0.0003, weight_decay=0.0001, eps=1e-08, betas=(0.9, 0.97), base_theta=10000.0, scale_factor=1.0, dtype=torch.bfloat16, fused_optimizer=True)


In [3]:
model = GPT(config)
model.to(torch.bfloat16)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(100288, 1024)
    (h): ModuleList(
      (0-15): 16 x Block(
        (ln_1): RMSNorm()
        (attn): CausalSelfAttention(
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (ln_2): RMSNorm()
        (mlp): MLP(
          (gate_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (up_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=100288, bias=False)
)

In [4]:
# count number of parameters in terms of billions
num_params = sum([param.nelement() for param in model.parameters()])
num_params / 1e6

371.16416

In [5]:
# torch.save(model.state_dict(), "model.pth")

In [6]:
# import tiktoken

# gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")
# gpt4_tokenizer.n_vocab

In [7]:
# sample input
text = torch.randint(0, 100288, (1, 1024))
text.shape

torch.Size([1, 1024])

In [ ]:
output = model(text)
output[0].shape

In [ ]:
# import torch
# import torch.nn.functional as F
# from transformers import AutoTokenizer



# def generate(model,
#              tokenizer,
#              prompt: str,
#              n_tokens_to_gen: int = 200,
#              sample: bool = True,
#              top_k: int = 40):
#     model.eval()

#     input_ids = tokenizer(prompt, return_tensors='pt').input_ids

#     for token_n in range(n_tokens_to_gen):
#         with torch.no_grad():
#             indices_to_input = input_ids
#             next_token_logits, loss = model(indices_to_input)
#             next_token_logits = next_token_logits[:, -1]

#         probs = F.softmax(next_token_logits, dim=-1)
#         (batch, vocab_size) = probs.shape

#         if top_k is not None:
#             (values, indices) = torch.topk(probs, k=top_k)
#             probs[probs < values[:, -1, None]] = 0
#             probs = probs / probs.sum(axis=1, keepdims=True)

#         if sample:
#             next_indices = torch.multinomial(probs, num_samples=1)
#         else:
#             next_indices = torch.argmax(probs, dim=-1)[:, None]

#         input_ids = torch.cat([input_ids, next_indices], dim=1)

#     output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

#     return output_completions